___
# ***Tarea 2.2 - Simulación de escenarios futuros***
___
***Instrucciones:***

Importar los precios de *Amazon* (cuyo ticker es: `'AMZN'`) desde Yahoo finance de los últimos 10 años y simular cien mil escenarios de los próximos 30 días para predecir la probabilidad de que el precio actual supere el 8%.



In [46]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import yfinance as yf

# Parámetros de la simulación
ticker = 'AMZN'
N_dias_simulacion = 30
N_escenarios = 100000
aumento_objetivo = 1.08  # 8%

# Definición de Fechas
fecha_fin_str = "2025-11-04"
fecha_inicio_str = "2015-11-04" # 10 años

print(f"Iniciando simulación para {ticker}...")
print(f"Período histórico: {fecha_inicio_str} al {fecha_fin_str}")
print(f"Escenarios: {N_escenarios:,}, Días a simular: {N_dias_simulacion}\n")

# Descargamos los datos (puede incluir la fila de hoy con NaN)
data_raw = yf.download(ticker, start=fecha_inicio_str, end=fecha_fin_str)['Close']

# Eliminar filas Nulas (como la de 'hoy' si no ha cerrado o es NaN)
data = data_raw.dropna()

# Obtener el último precio conocido (P0)
# Ahora iloc[-1] será el último día de cierre VÁLIDO
P0 = data.iloc[-1]
fecha_P0 = data.index[-1].strftime('%Y-%m-%d')
print(f"Datos históricos cargados. Último precio conocido (P0): ${P0.iloc[0]:.2f} (del {fecha_P0})")

# Calcular retornos porcentuales (deltas) y sus estadísticas
delta = data.pct_change().dropna()
media = delta.mean()
std = delta.std()

# Simular deltas futuros
delta_sim = np.random.normal(media, std, (N_dias_simulacion, N_escenarios))
delta_sim_df = pd.DataFrame(delta_sim)

# Proyectar precios futuros
# P_k = P_0 * prod(delta_i + 1)
precios_simulados = P0.iloc[0] * ((delta_sim_df + 1).cumprod())

# Obtener los precios finales (día 30)
precios_finales = precios_simulados.iloc[-1]

# Definir el precio objetivo
precio_objetivo = P0.iloc[0] * aumento_objetivo

# Calcular la probabilidad
B = precios_finales.values > precio_objetivo
probabilidad = B.mean()

print(f"Precio objetivo (+8%): ${precio_objetivo:.2f}")
print(f"Probabilidad de que el precio supere el 8% en 30 días: {probabilidad:.2%}")
print("Simulación completada.")

/tmp/ipython-input-732602967.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_raw = yf.download(ticker, start=fecha_inicio_str, end=fecha_fin_str)['Close']
[*********************100%***********************]  1 of 1 completed

Iniciando simulación para AMZN...
Período histórico: 2015-11-04 al 2025-11-04
Escenarios: 100,000, Días a simular: 30

Datos históricos cargados. Último precio conocido (P0): $254.00 (del 2025-11-03)


Precio objetivo (+8%): $274.32
Probabilidad de que el precio supere el 8% en 30 días: 32.55%
Simulación completada.
